In [1]:
import pandas as pd

Задание 1

In [5]:
ratings = pd.read_csv('Python_13_join1/Python_13_join/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
ratings_filtered_userId=ratings.groupby('userId').count()
ratings_filtered_userId.head(5)

,movieId,rating,timestamp
userId,,,
1,20,20,20
2,76,76,76
3,51,51,51
4,204,204,204
5,100,100,100


In [7]:
ratings_filtered_userId=ratings_filtered_userId[ratings_filtered_userId['rating']>100]
ratings_filtered_userId.head()

,movieId,rating,timestamp
userId,,,
4,204,204,204
8,116,116,116
15,1700,1700,1700
17,363,363,363
19,423,423,423


In [8]:
ratings_filtered_userId=ratings_filtered_userId.reset_index()
ratings_filtered_userId.head()

,userId,movieId,rating,timestamp
0,4,204,204,204
1,8,116,116,116
2,15,1700,1700,1700
3,17,363,363,363
4,19,423,423,423


In [9]:
rating_movie_fan=ratings[ratings['userId'].isin(ratings_filtered_userId['userId'].tolist())]
rating_movie_fan.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [10]:
rating_movie_fan_timestamp=rating_movie_fan.groupby('userId').agg({'timestamp': ['min', 'max']})
rating_movie_fan_timestamp.head()

timestamp            
               min         max
userId                        
4        949778714   949982274
8       1154389340  1154474527
15       997937239  1469330735
17      1127468587  1127476640
19       855190091   855195373

In [11]:
import time

In [12]:
def Lifetime(row):
    Lft= int(row['timestamp']['max'])-int(row['timestamp']['min'])
    return time.localtime(Lft).tm_mday

In [13]:
rating_movie_fan_timestamp['lifetime']=rating_movie_fan_timestamp.apply(Lifetime, axis=1)
rating_movie_fan_timestamp.head()

timestamp             lifetime
               min         max         
userId                                 
4        949778714   949982274        3
8       1154389340  1154474527        2
15       997937239  1469330735        9
17      1127468587  1127476640        1
19       855190091   855195373        1

Задание 2

In [14]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [15]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [16]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [17]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [25]:
table_1=(rzd.merge(auto, how = 'outer', on = 'client_id').fillna(0)
 .merge(air, how = 'outer', on = 'client_id').fillna(0))
table_1

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [26]:
table_2=table_1.merge(client_base, how = 'outer', on = 'client_id')
table_2

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


In [ ]:
#видимо я как-то не так поняла второе задание, потому что это же можно было сделать одной записью

Задание 3

Из идей:
-обучить модель, которая группирует часто встречающиеся у пользователя координаты, промечая их как "дом" (0/1), "работа/учеба" (0/1), "в дороге" (0/1), "прочее" (0/1) в зависимости от времени суток;

-если был визит без факта покупки, можно ставить флаг 0; если визит с покупкой, флаг 1;

Хранить в виде таблицы:
|Время визита (datetime))|id пользователя|id сессии|ширина|долгота|группировка координат


Использовать в таблице:
|Время визита (datetime)|id пользователя|id сессии|действие (визит/попупка)|дом|работа/учеба|в дороге|прочее|

Использовать эти данные для определения ситуации, когда пользователь просто заходит в приложение, а когда заходит с целью покупки.

Ключ для связывания |id сессии|

Если рассматривать в контексте интернет-магазина с продуктами, то координаты, ко время испольщования приложения могут помочь в следующем кейсе:
-если заказа из дома - заказанные продукты формируются и доставляются к указанному времени;
-если заказ из работы, то заказ формируется, но доставка осуществляется к дому из рассчета, в какое время пользователь обычно возвращается к координатам дом. 
По-идее это потимизирует работу доставки, сократит время ожидания. 